In [1]:
from molecule import atom
from molecule import vector
from molecule import gaussian
from molecule import molecule
from morsePotential import morsePotential
from notebookImporter import importNotebook
from IPython.display import clear_output

HartreeFock = importNotebook("Hartree_Class")

#set up the automated calculations used to generate the surface potential

#User Specified Variables

#Enter the atomic number for atoms 1 and 2
atom1 = 1
atom2 = 1

#Enter the number of electrons for atom 1 and 2
#Note the total number of electrons must be a multiple of 2, 
#otherwise, the computation will round the total number of electrons 
#down to the nearest multiple of 2
atomN1 = 1
atomN2 = 1

#Select a basis, for a list of available basis sets, please see the "basisSets" folder
basisName = "DEF2-SVP"

#Enter Starting and End Bond distance for the diatomic molecule
startR = .9
endR = 2

#Please enter the distance the bond distance should increase each iteration
#The smaller the number, the more accurate and time-consuming the computation will be
step = 0.05

#Please enter how small the difference in energy between two ground state energy calculations
#must be in order to convergence to occur
convergenceCritera = pow(10, -10)

#Please enter the maximum number of iterations the SCF procedure can take before
#the computation will fail as being unable to converge
maxIterations = 100

#System Defined Variables
E = []
R = []
basisSets = []
Xs = []
MOEnergy = []
delta = (endR - startR) / step
currentR = startR

print()

while(not(currentR >= endR)):
    
    #define the molecular system
    system = molecule()
    system.addAtom(atom(vector(1,1,2+currentR), atom1, atomN1))
    system.addAtom(atom(vector(1,1,2), atom2, atomN2))

    system.addBasis(basisName)
    
    #Set up the Hartree Procedure
    HF = HartreeFock.HF(system, convergenceCritera, maxIterations)

    #Store Bond distance, associated Energy, and MO Energies
    R.append(currentR * pow(10, -10))
    E.append(HF.SCF() * 2.19 * pow(10, 5))
    MOEnergy.append(HF.MOEnergy)
    basisSets.append( system.getBasis() )
    Xs.append( HF.X )
 
    #clear_output()
    print("Hartree-Fock Computation: " + str(currentR * 100 // endR) + "%")
    
    #update the current intenuclear distance
    currentR += step
    
print("Hartree-Fock Computation Complete")


Hartree-Fock Computation: 45.0%
Hartree-Fock Computation: 47.0%
Hartree-Fock Computation: 50.0%
Hartree-Fock Computation: 52.0%
Hartree-Fock Computation: 55.0%
Hartree-Fock Computation: 57.0%
Hartree-Fock Computation: 60.0%
Hartree-Fock Computation: 62.0%
Hartree-Fock Computation: 65.0%
Hartree-Fock Computation: 67.0%
Hartree-Fock Computation: 70.0%
Hartree-Fock Computation: 72.0%
Hartree-Fock Computation: 75.0%
Hartree-Fock Computation: 77.0%
Hartree-Fock Computation: 80.0%
Hartree-Fock Computation: 82.0%
Hartree-Fock Computation: 85.0%
Hartree-Fock Computation: 87.0%
Hartree-Fock Computation: 90.0%
Hartree-Fock Computation: 92.0%
Hartree-Fock Computation: 95.0%
Hartree-Fock Computation: 97.0%
Hartree-Fock Computation Complete


In [2]:
#Setup system for graphing purposes
import math
import scipy.integrate as integrate
import scipy.optimize as optimize
import numpy as np
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

#prepare morse potential for graphing
#morse = morsePotential(R, E)

#create a curve fitted morsePotential of the data
#uses the least squares method to fit the potential via SciPy
#def morsePotential(r, r0, A, D, a1, a2, a3):
def morsePotential(r, r0, A, D, a):
    return A + D*pow( (1-np.exp(-(a*(r-r0)))), 2) 
    #return D * np.exp(-2*a*(r-r0)) - 2*D*np.exp(-a*(r-r0)) + c 
    #return a0 * (a1 * (r - r0) + a2 * pow((r - r0), 2) + a3 * pow(r-r0, 3)) + a4
    #return (a1 * np.exp(-2 * a2 * ( r - r0)) - (2 * a1 * np.exp(-a2 * ( r - r0)))) + c
    #return (a1 * np.exp(-a2 * r) * (1 - (a3*r))) - (a4 / (pow(r,6) + (a5*pow(r,-6)))) 
    #return A - (D*( 1 + a1*(r-r0) + a2*pow((r-r0),2) + a3*pow(r-r0, 3) ) * np.exp(-a1*(r-r0)))
    
popt, pcov = optimize.curve_fit(morsePotential, R, E)

print(popt)
print(pcov)

#Prepare data for graphing
minEIndex = E.index(min(E))
minE = E.pop(minEIndex)
optimalR = R.copy().pop(minEIndex)
#basisSet = basisSets[ minEIndex ]
#X = Xs[minEIndex]

[ 2.26691423e+00  2.62407134e+04 -8.65348778e+00  2.28286808e+00]
[[-9.23342676e+09  1.53086370e+16 -7.98209212e+11  1.62045431e+09]
 [ 2.93715320e+16  3.31131474e+22  5.49965267e+17  1.15311964e+16]
 [-1.44217615e+12  4.01929631e+17 -6.27162440e+13  1.92014085e+11]
 [-3.16904729e+09  2.19400102e+16 -3.35043034e+11  1.26966187e+10]]


#morseData = morse.computePotential()
#w = (morse.a / (2 * math.pi)) * pow((2 *  morse.D / u), 0.5)
#print(w)

#k = 4 * math.pi * math.sqrt(2 * morse.D) / morse.a
#print("K: " + str(k))

ve = []
#for n in range(18):
#    #ve.append( w * ((n + .5) - pow((n+.5),2)) )
#    ve.append( -morse.D + (w * (n + .5)) - ( (pow(w,2) /  (4*morse.D)) * pow((n + .5), 2) )  )
#print(ve)


#print optimized bond distance
print("Optimal Bond Distance: " + str(R[minEIndex]))


size = 5
basisSet = []
u = 0.5
r0 = popt[1]
def newPlaneWave(k):
    
    def planeWave(x):
       # print("||"*20)
       # print(x)
        return math.cos(k* math.pi * (x-r0)) + math.sin(k * math.pi * (x))*1j 
    
    return planeWave

    
class basisFunction:
    
    k = 0
    f = 0
    normalization = 0
    
    def __init__(self, f, k, allSpace=1):
        
        self.k = k
        self.f = f 
        self.normalization = math.sqrt(1 / allSpace)
    
    def compute(self, x):
        #print(self.normalization)
        return self.normalization * self.f(x)
    
    def compute2ndPD(self, x):
        return -self.f(x) * self.normalization
    

for n in range(size):
    basisSet.append( basisFunction(newPlaneWave((2*n + 1) ), (2*n + 1), 1) )
    
V = np.zeros([size, size], dtype = complex)
T = np.zeros([size, size], dtype = complex)
S = np.zeros([size, size], dtype = complex)

for index1, b1 in enumerate(basisSet):
    for index2, b2 in enumerate(basisSet):
        
        #base = lambda x : b1.compute(x).conjugate() * b2.compute(x)
        #real = lambda x : base(x).real
        #imag = lambda x : base(x).imag 
        
        SIntegrandR = lambda x : (b1.compute(x).conjugate() * b2.compute(x)).real
        SIntegrandI = lambda x :(b1.compute(x).conjugate() * b2.compute(x)).imag
        
        S[index1, index2] += integrate.quad(SIntegrandR, -50, 50, limit=1000)[0] + integrate.quad(SIntegrandI, -50, 50, limit=1000)[0]*1j
        
        VIntegrandR = lambda x : (b1.compute(x).conjugate() * morsePotential(x, popt[0], popt[1], popt[2], popt[3], popt[4]) * b2.compute(x)).real
        VIntegrandI = lambda x : (b1.compute(x).conjugate() * morsePotential(x, popt[0], popt[1], popt[2], popt[3], popt[4]) * b2.compute(x)).imag
        
        V[index1, index2] = integrate.quad(VIntegrandR, -50, 50, limit = 1000)[0] + integrate.quad(VIntegrandI, -50, 50, limit = 1000)[0]*1j
        
for index1, b1 in enumerate(basisSet):
    for index2, b2 in enumerate(basisSet):
        
        TIntegrandR = lambda x : (b1.compute(x).conjugate() * (1/(2 * u)) * b2.compute2ndPD(x)).real
        TIntegrandI = lambda x : (b1.compute(x).conjugate() * (1/(2 * u)) * b2.compute2ndPD(x)).imag
        
        T[index1, index2] = integrate.quad(TIntegrandR, -50, 50, limit=1000)[0] + integrate.quad(TIntegrandI, -50, 50, limit=1000)[0] * 1j

print()
print("S"*20)
print(S)
print()
print()
print(V)
print("------")
print(T)

H = T + V

print("*"*20)
print(H)


print("&"*5)
print(H)
EPW = np.linalg.eigh(H)[0]

print("|*)"*20)

for e in EPW:
    print(e)
    

size = 10
L = 1.41

m = 0.5
basis = []

def zeroError(x, tolerance=""):
    
    if(tolerance == ""):
        tolerance = pow(10, -14)
    
    if(abs(x) < tolerance):
        return 0
    else:
        return x
             
class basisFunction:
    
    n = 1
    L = 1
    normalization = 1
    
    def __init__(self, n, L):
        
        self.n = n
        self.L = L
        self.normalization = math.sqrt(2/L)

    def compute(self, x):
        return self.normalization * math.sin(self.n*math.pi*x / self.L)
    
    def compute2ndPD(self, x):
        return self.compute(x)

for n in range(size):
    basis.append( basisFunction(n+1, L) )
    
S = np.zeros([size, size])
V = np.zeros([size, size])
T = np.zeros([size, size])

for index1, b1 in enumerate(basis):
    for index2, b2 in enumerate(basis):
        
        SIntegrand = lambda x : b1.compute(x) * b2.compute(x)

        S[index1, index2] += zeroError(integrate.quad(SIntegrand, 0, L)[0])

        VIntegrand = lambda x : b1.compute(x) * morsePotential(x, popt[0], popt[1], popt[2], popt[3], popt[4]) * b2.compute(x)
        V[index1, index2] += zeroError(integrate.quad(VIntegrand, 0, L)[0])
        
        TIntegrand = lambda x : b1.compute(x) * (1/(2*m)) * b2.compute2ndPD(x)
        T[index1, index2] += zeroError(integrate.quad(TIntegrand, 0, L)[0]) 
        
print(S)
print("V"*40)
print(V)
print("T"*50)
print(T)

H = V + T
print("H"*70)
print(H)

PIB = np.linalg.eigh(H)[0]

In [3]:
from scipy.misc import derivative as ddx

u = 1
w = 1
re = 1
test = 1 

#d, r0, a

#Returns a lambda function of the nth hermite polynomial
def hermite(n, k):
    
    c = pow(-1, k) * math.factorial(n) / ( math.factorial(k) * math.factorial(n - 2*k) )
        
    if(k == 0):
        return lambda r : c * pow(2*math.sqrt(u*w)*( (r-re) * test ), n-(2*k))
    else:
        return lambda r : c * pow(2*math.sqrt(u*w)*( (r-re) * test ), n-(2*k)) + hermite(n, k-1)(r)

def buildHermite(n):
    
    return hermite(n, n // 2)

#returns the normalization constant for the Harmonic Oscillator Function
def C(n):
    
    return math.sqrt( math.sqrt(u*w*pow(test,2)) /  (pow(2, n) * math.factorial(n) * np.sqrt(math.pi)) ) 

#Creates a new Harmonic Oscillator function
#and returns it as a lambda function
def newHO(n):
    
    return lambda r : C(n) * buildHermite(n)(r) * np.exp(-u * w * pow((r-re) * test,2) / 2)

basisSize = 5
basisSet = []
S = np.zeros([basisSize, basisSize])
V = np.zeros([basisSize, basisSize])
T = np.zeros([basisSize, basisSize])

Tints = []
Vints = []

#Build the basis Fucntions
for i in range(basisSize):
    basisSet.append( newHO(i) )
    
#Verify orthonormality of the basis set
for i in range(basisSize):
    for j in range(basisSize):
        
        integrand = lambda r : basisSet[i](r) * basisSet[j](r)
        
        S[i,j] += integrate.quad(integrand, -np.inf, np.inf, limit=10000, epsabs=pow(10, -50), epsrel=pow(10, -30))[0]
        
        if( (i == j and abs(S[i, j] - 1) >= .1 or not 1==j and abs(S[i,j]) >= pow(10, -14) )):
            print("Overlap Error: i=" + str(i) + ", j=" + str(j))
            print(S[i,j])
            
        integrand = lambda r : basisSet[i](r) * morsePotential(r, popt[0], popt[1], popt[2], popt[3]) * basisSet[j](r)
        
        V[i, j] += integrate.quad( integrand, -40, 40)[0]#, limit=10000, epsabs=pow(10, -50), epsrel=pow(10, -30))[0]
 
        integrand2 = lambda r : basisSet[i](r) * (-1/(2*.5)) * ddx(basisSet[j], r, n=2)

        T[i, j] += integrate.quad(integrand2, -40, 40)[0]
    
        Tints.append(integrand2)
        Vints.append(integrand)
    
        #if(i == j):
        #    T[i,j] += 2*i + 1
        #elif (i == j + 2):
        #    T[i,j] += -math.sqrt( i*(i-1) )
        #elif (i == j - 2):
        #    T[i,j] += -math.sqrt((i+1) * (i+2))
        #else:
        #    T[i,j] = 0
#T *= w / 4        
print()
print("Overlap Matrix: ")
print()
print(S)
print()
print("*" * 40)
print()

print("V")
print()
print(V)
print()
print("*"*40)
print()

print("T")
print()
print(T)
print()
print("*" * 40)
print()

H = T + V
print(H)
print()
EHO = np.linalg.eig(H)[0]
print(EHO)

print(len(basisSet))
######################################################################################

#y3 = lambda x : basisSet[4](x) * morsePotential(x, popt[0], popt[1], popt[2] ) *  basisSet[2](x)
#dx = 0.01
#RI = int(6 / dx)
#print(RI)
figure = { 
    "data": [
        
        #create the Hartree-Fock generated Potential Energy Surface
        {
    #        "type":"scatter",
    #        "x":[x * dx for x in range(-RI, RI)],
    #        "y":[y3(x * dx) for x in range(-RI, RI)],
    #        "connectgaps":False,
    #        "mode":"markers", 
    #        "name":"Hartree-Fock Computed",
    #        "marker":{"color":"blue"}
        }
    ]
}

#iplot(figure)



A:\Apps\Anaconda\lib\site-packages\ipykernel_launcher.py:54: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.



Overlap Error: i=0, j=0
0.9999999999999999
Overlap Error: i=2, j=2
0.9999999999999998
Overlap Error: i=3, j=3
1.0
Overlap Error: i=4, j=4
1.0

Overlap Matrix: 

[[ 1.00000000e+00  0.00000000e+00  2.77555756e-17  0.00000000e+00
   1.38777878e-17]
 [ 0.00000000e+00  1.00000000e+00  1.38777878e-17  0.00000000e+00
   3.81639165e-17]
 [ 2.77555756e-17  1.38777878e-17  1.00000000e+00 -2.08166817e-17
   5.48606299e-17]
 [ 0.00000000e+00  0.00000000e+00 -2.08166817e-17  1.00000000e+00
   1.38777878e-17]
 [ 1.38777878e-17  3.81639165e-17  5.48606299e-17  1.38777878e-17
   1.00000000e+00]]

****************************************

V

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

****************************************

T

[[ 4.42398434e-01  1.26478960e-17 -5.50695315e-01 -3.52907807e-17
  -7.94860221e-02]
 [-6.32767493e-17  1.22119922e+00 -5.12827628e-17 -7.94860221e-01
   1.25957167e-16]
 [-5.50695315e-01 -2.84060969e-17  1.80529980e+00  6.9388939

In [4]:
#EHO = HF.vibrationalEnergy(lambda x : morsePotential(x, popt[0], popt[1], popt[2], popt[3], popt[4]), 0.5)
#print(EHO)

In [5]:
#prepare data for use with graphing
dx = .1
steps = 15/dx

data = [x*dx for x in range(int(steps))]

figure = { 
    "data": [
        
        #create the Hartree-Fock generated Potential Energy Surface
        {
            "type":"scatter",
            "x":R,
            "y":E,
            "connectgaps":False,
            "mode":"markers", 
            "name":"Hartree-Fock Computed",
            "marker":{"color":"blue"}
        },
    
        #Highlight the minimum energy point in red
        {
            "type":"scatter",
            "x":[optimalR],
            "y":[minE],
            "name":"Optimal Bond Distance",
            "marker":{"color":"red"}
        },
        
        #Create and plot the Morse Potential fit
        {
            "type":"scatter",
            "x":data,
            "y": [morsePotential(r, popt[0], popt[1], popt[2], popt[3]) for r in data] ,
            "connectgaps":False,
            "name":"Morse Potential Approximation",
            "marker":{"color":"green"}
        },
        
        {
            "type":"scatter",
            "x":[popt[1]] * len(EHO),
            "y":EHO,
            "connectgaps":False,
            "name":"Harmonic Oscillator",
            "marker":{"color":"blue"}
        },
        
        {
            "type":"scatter",
            "x":[x for x in range(-15, 15)],
            "y":[integrand(x) for x in range(-15, 15)],
            "connectgaps":True,
            "name":"V Integrand",
            "marker":{"color":"red"},
        }, 
        {
           "type":"scatter",
            "x":[x for x in range(-15, 15)],
            "y":[integrand2(x) for x in range(-15, 15)],
            "connectgaps":True,
            "name":"T Integrand",
            "marker":{"color":"purple"}, 
        },
        
        {
           "type":"scatter",
            "x":[x for x in range(-15, 15)],
            "y":[integrand2(x) for x in range(-15, 15)],
            "connectgaps":True,
            "name":"T Integrand",
            "marker":{"color":"purple"}, 
        }
        #{
        #    "type":"scatter",
        #    "x":[popt[1]] * len(EPW),
        #    "y":EPW,
        #    "connectgaps":False,
        #    "name":"Plane Wave",
        #    "marker":{"color":"blue"}
        #},
        
           #{
           # "type":"scatter",
           # "x":[PIB] * len(EPW),
           # "y":PIB,
           # "connectgaps":False,
           # "name":"PIB Solution",
           # "marker":{"color":"blue"}
        #}
        
    ],
    
    #Set up the layout of the graph
    "layout":
        {
           "xaxis":{"title":"Bond Distance in Atomic Units"},
           "yaxis":{"title":"Energy in Hartrees"},
            "title":{"text":"Hartree-Fock Energy VS Bond Distance"}
        },    
}

#Loop over all MO Energies
#for vibrationalEnergy in MOEnergy[minEIndex]:
#        figure["data"].append(
#            {
#                "type":"scatter",
#                "x":[1.4],
#                    "y" : [vibrationalEnergy]
#            }
#        )    

#for i in range(basisSize):
#    for j in range(basisSize):
#        figure["data"].append({"type":"scatter",
#            "x":[x for x in range(-15, 15)],
#            "y":[Tints[0](x) for x in range(-15, 15)],
#            "connectgaps":True,
#            "name":"T Integrand"+ str(i) + "," + str(j)}
#        )
    
#        figure["data"].append(
#            {"type":"scatter",
#            "x":[x for x in range(-15, 15)],
#            "y":[Vints[0](x) for x in range(-15, 15)],
#            "connectgaps":True,
#            "name":"V Integrand " + str(i) + "," + str(j),
#            },
#    )

iplot(figure)